In [1]:
import os
import textract

In [2]:
pdf_path = '../data/drapeau'

pdfs = []
for f in os.listdir(pdf_path):
    if os.path.isfile(os.path.join(pdf_path, f)):
        pdfs.append(f)
pdfs[:10]

['all.txt',
 'juuu.txt',
 'KB_JB1051_1921-10-08_01-00001.txt',
 'KB_JB1051_1921-10-15_01-00001.txt',
 'KB_JB1051_1921-10-15_01-00003.txt',
 'KB_JB1051_1921-10-15_01-00004.txt',
 'KB_JB1051_1921-10-22_01-00003.txt',
 'KB_JB1051_1921-10-22_01-00004.txt',
 'KB_JB1051_1921-12-03_01-00004.txt',
 'KB_JB1051_1921-12-17_01-00004.txt']

## Créer un répertoire TXT s'il n'existe pas encore

In [3]:
txts = []
for f in os.listdir(txt_path):
    if os.path.isfile(os.path.join(txt_path, f)):
        txts.append(f)
txts[:10]

NameError: name 'txt_path' is not defined

In [ ]:
with open("../data/drapeau/all.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(txt_path):
        if file.endswith(".txt"):
            with open(os.path.join(txt_path, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())


# Extraction de Keywords : avec le corpus idependance qui correspond à independance belge 

In [ ]:
import os
import yake

## Extraire les mots clés d'un document avec Yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

## ici nous prenons idependance , pour avoir le corpus du soir et drapeau , remplaçons idependance pas celle qui convient

In [ ]:
# Lister les Fichiers
data_path = "../data/idependance/"
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]

In [ ]:
# Imprimer le nombre de fichiers identifiés
len(files)

In [ ]:
years = [str(year) for year in range(1831, 1840)]  # Par exemple, de 1996 à 2022

for year in years:
    # Filtrer les fichiers correspondant à l'année actuelle
    files_for_year = [f for f in os.listdir(data_path) if year in f]


In [ ]:
for year in years:
    # Filtrer les fichiers correspondant à l'année actuelle
    files_for_year = [f for f in os.listdir(data_path) if year in f]

    # Initialiser le texte pour l'année en cours
    text = ""

    # Concaténer le contenu de tous les fichiers correspondant à l'année
    for filename in files_for_year:
        file_path = os.path.join(data_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            text += f.read()

    # Affichage des premiers 500 caractères du texte pour chaque année
    print(f"Texte pour l'année {year} :")
    print(text[:500])

In [ ]:
keywords = kw_extractor.extract_keywords(text)
keywords

In [ ]:
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

In [ ]:
import csv

# Création d'une liste pour tous les mots-clés de 1831 à 1835
all_keywords_1831_1840 = []

# Faire la même opération sur tous les documents de la période spécifiée
for year in range(1831, 1840):
    files_for_year = [f for f in os.listdir(data_path) if str(year) in f]

    for filename in files_for_year:
        text = open(os.path.join(data_path, filename), 'r', encoding='utf-8').read()
        keywords = kw_extractor.extract_keywords(text)
        kept = []
        for kw, score in keywords:
            words = kw.split()
            if len(words) == 2:
                kept.append(kw)
                all_keywords_1831_1840.append((kw, score))
        print(f"{filename} mentions these keywords: {', '.join(kept)}...")

# Tri des mots-clés par score (croissant)
all_keywords_1831_1840.sort(key=lambda x: x[1])

# Écriture des mots-clés dans un fichier CSV
with open('keywords_1831_1840.csv', 'w', newline='', encoding='utf-8') as csvfile:
    keyword_writer = csv.writer(csvfile, delimiter=',')
    keyword_writer.writerow(['Keyword', 'Score'])  # Écriture de l'en-tête
    for kw, score in all_keywords_1831_1840:
        keyword_writer.writerow([kw, score])

# Reconnaissance d'entités nommées avec SpaCy : avec le corpus idependance

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
n=100000
text = open("../data/keywords_1831_1840.csv", encoding='utf-8').read()[:n]

In [ ]:
# Charger le texte
n=100000
text = open("../data/keywords_1831_1840.csv", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

 lister  les organisations (PER) les plus mentionnées dans le corpus

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

lister  les organisations (ORG) les plus mentionnées dans le corpus

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "ORG" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

lister  les LOCALISATIONS (LOC) les plus mentionnées dans le corpus

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

# Nuages de mots

## Imports et stopwords

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem","III","ETC","dès","elles","et","pour","seul","celui","alors","vers","effet","ailleurs",
       "tel","deja","culs","chez","l'","d'","bur'","faç'","mais","ces","leurs","meme","aux'","sur'","sans'","moins", "cas", "vue", "toujour", "ceux",
       "peu","moment","nos","seul","avoir","été","point","ici","quand","plus","fut","non","notre",]
sw = set(sw)

## Créer un fichier contenant le texte de tous les journaux d'une année donnée

In [ ]:
# Définir les chemins d'accès
data_path = '../data'
txt_path = '../data/idependance'  # Ou un chemin spécifique à vos fichiers

# Liste des années de 1835 à 1840
years = [str(year) for year in range(1835, 1840)]  # 1953 exclu pour inclure 1952

# Réaliser le processus pour chaque année
for year in years:
    # Stocker le contenu des fichiers dans une liste
    content_list = []
    for txt in os.listdir(txt_path):
        if year in txt:
            with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
                content_list.append(f.read())



In [ ]:
# Écrire tout le contenu dans un fichier temporaire
temp_path = f'../data/tmp_{year}'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Fonction pour nettoyer le texte
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        sw = set(nltk.corpus.stopwords.words('english'))  # Ajoutez les stopwords appropriés ici
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

# Nettoyer le texte pour l'année donnée
clean_text(year, folder=temp_path)
# Vérifier le résultat après le nettoyage
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

# Afficher les 500 premiers caractères après nettoyage
print(after[:500])

In [ ]:
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

## Nettoyer le fichier à l'aide d'une fonction de nettoyage

In [ ]:
# Compter les fréquences des mots
frequencies = Counter(after.split())

### Créer, stocker et afficher le nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

# Sentiment analysis 

## 1. Textblob-FR

Documentation: https://textblob.readthedocs.io/en/dev/

### Imports

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

### Création d'une fonction `get_sentiment`

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Analyser les sentiments des phrases suivantes:

In [ ]:
get_sentiment("je demande à êlre jugé, s'il y a possibilité de me juger; si la possibilité de méjuger n'existe plus, déclarez-le")

In [ ]:
get_sentiment("Partant de là , ou doit donner au mot de domination le sens le plus large , el reconnailre que la Belgique toule entière a de droit changé défini ivemenl de domination aux yeux des puissances qui l'ont reconnue, aux yeux du Roi Guillaume, qui a abdiqué , quant à la Belgique.")

In [ ]:
get_sentiment("Nous soutenons, au conlraire, que l'amnistie doit recevoir l'application la plus générale ; qne l'amnistie doit profiter à tous ceux qui , soil dans le Limbourg belge, ou le Limbourg hollandais, soit dans le Luxembourg allemand, soit dans le Luxembourg belge , soit enfin dans la Belgique entière, se irouveraienl dans le cas d'être inquiétés pour une parlicipation quelconque aux fails de la révolution")

In [ ]:
get_sentiment("l est bien certain que s'il y avait eu dans ce fait récompense , il n'y aurail pas eu encouragement ponr la discipline ; mais cela n'est nullement exact")

In [ ]:
get_sentiment("En effet, pour quiconque veut lire l'article 20 avec impartialité, sans préoccupation, sans nécessité de falsifier son sens, ce sens est de loule évidence.")

In [ ]:
get_sentiment("je vous signalais dernièrement la rréàiion du conseil dé l'EmpixejReichsrelhi, fà- ranti par la Charte et qui vient d'être organisé comme une mesure de la plus haute importance, si, Oh effet, on y procède d'après la lettre et l'esprit dé la Charte qui dit, art. 96 el 97 : « Qu'à côlé de la couronne et du pouvoir exécutif, il est institué un conseil do l'Empire ayant voix consultative stir toutes les aîalr'es sur lesquelles le pouvoir exécutif réclamera son avis ; que les membres de ce conseil sont nommés par l'Empereur; et qde dans leur nomination on doit, autant que possible, prendre en considération les diverses parties dé i'empifè (les diverses nationalités) » Par là on arrivera d'autant plus aisément à réunir les plus grandes capacités dans l'administration, la politique et la jurisprudence, que leur position dans le conseil de l'Empire n'est pas comme celle du ministère responsable et ne représente pas une certaine couleur")

In [ ]:
get_sentiment("Je profite de la trêve que le dimanche vient d'apporter aux débats de la semaine, pour ajouter encore quelques mots aux détails que je vous ai transmis dans ma lettre précédente sur la discussion générale du budget. Les débats ont eu, comme dej-aison, un caractère double Ils ont roulé en même temps sur les mérites du budget proposé et sur la politique générale du ministère.")

In [ ]:
get_sentiment("Mais je.crois que la 'Chambre poserait un précédent fàoheux en l'admttânt à voter")

In [ ]:
get_sentiment("L’équipe anversoise est arrivée A ses fins, aprè*. avoir mené le match quasi de bout en bout")

In [ ]:
get_sentiment("“Van Kersschaever. au pivot, n’atteignit Jamais son rendement habituel Alors qu’il était considéré comme l’atout majeur de l’équipe ostendnise, il se révéla franchement décevant et il ne tira guère profit, hormis en récupération défensive, de son avantage de taille")

# clustering

In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

In [ ]:
import nltk

nltk.download('punkt')

ici on utilise le corpus drapeau c'est pareil pour le soir et independance belge

In [ ]:
data_path = "../data/le soir/"

choisir une periode

In [ ]:
years_1939_to_1945 = [str(year) for year in range(1939, 1945)]  # Liste des années de 1831 à 1835

articles_1939_to_1945 = [f for f in os.listdir(data_path) if any(year in f for year in years_1939_to_1945)]

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [ ]:
years_1939_to_1945 = [str(year) for year in range(1939, 1945)]  # Liste des années de 1831 à 1835

articles_1939_to_1945 = [
    open(os.path.join(data_path, f), "r", encoding="utf-8").read() 
    for f in os.listdir(data_path) 
    if any(year in f for year in years_1939_to_1945)
]

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(articles_1939_to_1945)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
tfidf_array = tfidf_vectors.toarray()

### Définir un nombre de clusters

In [ ]:
N_CLUSTERS = 7

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [ ]:
files = os.listdir(data_path)  # Obtenez la liste de tous les fichiers dans le répertoire

# Assurez-vous de la correspondance entre les fichiers et les vecteurs TF-IDF
# Cela suppose que les fichiers sont dans le même ordre que les vecteurs TF-IDF ont été créés
clustering = collections.defaultdict(list)
for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

## Visualiser les clusters

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

silhouette score moyen

In [ ]:
from sklearn.metrics import silhouette_score

# Supposons que reduced_vectors est votre ensemble de données réduites après PCA, et clusters contient les labels de clusters attribués à chaque point

# Calcul du score de silhouette
silhouette_avg = silhouette_score(reduced_vectors, clusters)
print(f"Silhouette Score moyen : {silhouette_avg}")

# Visualisation des données réduites avec les clusters et les centroïdes
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

Similarité moyenne intra-cluster pour le cluster

In [ ]:
clustered_texts = {}

from sklearn.metrics.pairwise import cosine_similarity

# Supposons que 'clustered_texts' est une liste de clusters où chaque cluster contient des documents similaires
articles_1939_to_1945 = [
    open(os.path.join(data_path, f), "r", encoding="utf-8").read() 
    for f in os.listdir(data_path) 
    if any(year in f for year in years_1939_to_1945)
]

# Utilisez ces articles pour créer la variable all_articles
all_articles = articles_1939_to_1945
# Création des clusters et stockage des textes associés
for idx, cluster_idx in enumerate(clusters):
    if cluster_idx not in clustered_texts:
        clustered_texts[cluster_idx] = []
    clustered_texts[cluster_idx].append(all_articles[idx])

# Calcul de la similarité intra-cluster pour chaque cluster
for cluster_idx, cluster_text in clustered_texts.items():
    # Convertir le cluster de texte en une matrice TF-IDF
    tfidf_matrix_cluster = vectorizer.fit_transform(cluster_text)

    # Calculer la similarité entre les documents dans le cluster
    similarity_matrix = cosine_similarity(tfidf_matrix_cluster, tfidf_matrix_cluster)
    mean_similarity = similarity_matrix.mean()  # Similarité moyenne dans le cluster
    print(f"Similarité moyenne intra-cluster pour le cluster {cluster_idx} : {mean_similarity}")

rom sklearn.metrics.pairwise import cosine_similarity

# Supposons que 'clustered_texts' est une liste de clusters où chaque cluster contient des documents similaires
for cluster_idx, cluster_text in clustered_texts.items():
    # Convertir le cluster de texte en une matrice TF-IDF (exemple)
    tfidf_vectorizer_cluster = TfidfVectorizer()
    tfidf_matrix_cluster = tfidf_vectorizer_cluster.fit_transform(cluster_text)

    # Calculer la similarité entre les documents dans le cluster
    similarity_matrix = cosine_similarity(tfidf_matrix_cluster, tfidf_matrix_cluster)
    mean_similarity = similarity_matrix.mean()  # Similarité moyenne dans le cluster
    print(f"Similarité moyenne intra-cluster pour le cluster {cluster_idx} : {mean_similarity}")

# Word Embeddings : le modèle Word2Vec

In [ ]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

dans ce code c'est le soir qui est utilisé

In [ ]:
infile = f"../data/le soir/sen (2).txt"
sentences = MySentences(infile)

### Détection des bigrams


In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

## Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

### Charger le modèle en mémoire

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

### Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("gouvernement", "politique")

In [ ]:
model.wv.similarity("revolution", "syndicat")

In [ ]:
model.wv.similarity("communisme", "socialisme")

In [ ]:
model.wv.similarity("diplomatie", "commission")

In [ ]:
model.wv.similarity("guerre", "manifestation")